In [18]:
import json
import random

In [83]:
with open("./data/info23.json", 'r', encoding="utf-8") as f:
    info = json.load(f)

In [84]:
def choose_space(info):
    i = 10

    # 通过操作数组顺序，选择场地号
    target_space = info.pop(i - 1) # 提取出目标场地
    random.shuffle(info) # 打乱列表的其余位置
    info.insert(0, target_space) # 将目标场地插入到首位


    for space in info:
        print(space)
        for key, value in space.items():
            if key.isnumeric() and value["reservationStatus"] == 1 and value["startDate"] in ["2024-06-20 21:30"]:
                # if reserver.n_site == 2:
                #     if str(int(key) + 1) in space.keys() and space[key + 1]["reservationStatus"] == 1:
                #         return [{
                #             "spaceId": str(space["id"]), "timeId": str(int(key) + 1), "venueSpaceGroupId": None
                #         }, {
                #             "spaceId": str(space["id"]), "timeId": str(key), "venueSpaceGroupId": None
                #         }]
                # else:
                return [{
                    "spaceId": str(space["id"]), "timeId": str(key), "venueSpaceGroupId": None
                }]
    return []

In [85]:
choose_space(info)

{'6980': {'tradeNo': 'D240304275779', 'orderId': None, 'reservationStatus': 4, 'orderFee': 0.0, 'startDate': '2024-06-20 13:30', 'endDate': '2024-06-20 14:30', 'useNum': 1, 'waitNum': 0, 'alreadyNum': 1, 'reservationSummary': None, 'adminTake': False, 'adminRemark': None, 'orderName': None, 'orderUid': None, 'orderRoleId': None, 'orderRoleIdStr': None, 'orderDetailId': '708785', 'orderType': 5, 'takeUp': False, 'takeUpExplain': None, 'buddyNum': None, 'gateInCount': None, 'isGateIn': None}, '6982': {'tradeNo': 'D240304275779', 'orderId': None, 'reservationStatus': 4, 'orderFee': 0.0, 'startDate': '2024-06-20 15:30', 'endDate': '2024-06-20 16:30', 'useNum': 1, 'waitNum': 0, 'alreadyNum': 1, 'reservationSummary': None, 'adminTake': False, 'adminRemark': None, 'orderName': None, 'orderUid': None, 'orderRoleId': None, 'orderRoleIdStr': None, 'orderDetailId': '708787', 'orderType': 5, 'takeUp': False, 'takeUpExplain': None, 'buddyNum': None, 'gateInCount': None, 'isGateIn': None}, '6981': {

[{'spaceId': '100', 'timeId': '6987', 'venueSpaceGroupId': None}]

In [30]:
info = list(range(1, 8))
print(info)

i = 7
target_space = info.pop(i - 1) # 提取出目标场地
random.shuffle(info) # 打乱列表的其余位置
info.insert(0, target_space) # 将目标场地插入到首位

print(info)

[1, 2, 3, 4, 5, 6, 7]
[7, 3, 5, 1, 6, 2, 4]


In [86]:
len(info)

20

In [99]:
int(random.randint(1, 2))

1